In [2]:
!pip install pytesseract -q
!sudo apt update
!sudo apt install tesseract-ocr

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://cli.github.com/packages stable InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,575 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,521 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,937 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa

In [3]:
!pip install langchain-community --upgrade
!pip install unstructured --upgrade
!pip install crewai --upgrade
!pip install crewai-tools --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 33.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.1 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223

In [41]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [5]:
from crewai import Agent, Task, Crew, Process

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [60]:
#pdf_path=r'/content/invoice-0-4.pdf'
pdf_path = r'/content/invoice_sample.pdf'

In [9]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 33.7 MB/s eta 0:00:00


In [31]:
# ----------------------
# PDF Extraction Function
# ----------------------
import pytesseract
from PIL import Image
import io
import fitz  # PyMuPDF

def extractor(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = []

    for page_number, page in enumerate(doc, start=1):
        text = page.get_text()
        if not text.strip():
            # Convert page to image and run OCR
            pix = page.get_pixmap()
            img_bytes = pix.tobytes(output="png")  # get PNG bytes
            img = Image.open(io.BytesIO(img_bytes))
            ocr_text = pytesseract.image_to_string(img)
            full_text.append(ocr_text)
        else:
            full_text.append(text)

    doc.close()
    return "\n".join(full_text)

# Example usage:
# text = extract_text_from_pdf("sample.pdf")
# print(text)


In [72]:
from crewai.tools import BaseTool

# Create a tool class for your extractor function
class ExtractorTool(BaseTool):
    name: str = "Invoice OCR Extractor"
    description: str = "A tool to extract all text content from an invoice PDF using advanced OCR methods."

    def _run(self, invoice_path: str) -> str:
        """
        The method that will be executed by the agent.
        It calls the custom extractor function.
        """
        return extractor(invoice_path)

# Instantiate the custom tool
ocr_extractor_tool = ExtractorTool()


In [34]:
#ocr_extractor_tool.run(pdf_path)

Using Tool: Invoice OCR Extractor


' \n \nBioplex \nwe love chemistry \n5 Rue Bader \nNarbonne, Aude, 11100 \nPhone: +33 140 260294 \nINVOICE \nINVOICE # BPXINV-00550 \nDATE: 23.05.2021 \nTO: \nRoger Bigot \nbonbono \n4 Rue des Cites \nAubervilliers, Seine-Saint-Denis, 93300 \nPhone: +33 148 340990 \nSHIP TO: \nRoger Bigot \nbonbono \n4 Rue des Cites \nAubervilliers, Seine-Saint-Denis, 93300 \nPhone: +33 148 340990 \nCOMMENTS OR SPECIAL INSTRUCTIONS: \nNone \nSALESPERSON \nP.O. NUMBER \nTERMS \nMarianne de la Guillaume \nBPXPO-00536 \nDue after 30 days \n \nQUANTITY \nDESCRIPTION \nUNIT PRICE \nTOTAL \n10 \nDextromethorphan polistirex \nBPXPN-00057 \n12.45 \n124.50 \n25 \nVenlafaxine Hydrochloride \nBPXPN-00012 \n16.00 \n400.00 \n25 \nMetoclopramide Hydrochloride (BPXPO-00537) \nBPXPN-00002 \n9.99 \n249.75 \n10 \nAvobenzone, octinoxate (BPXPO-00538) \nBPXPN-00027 \n4.45 \n44.50 \n10 \nVerapamil hydrochloride \nBPXPN-00066 \n7.89 \n78.90 \n15 \nTiagabine hydrochloride \nBPXPN-00017 \n10.25 \n153.75 \n10 \nZiprasidone hyd

In [80]:
from crewai import Agent, Task, Crew, Process

# Define Agents
ocr_agent = Agent(
    role="OCR Specialist",
    goal="Extract all text content from an invoice PDF using advanced OCR methods.",
    backstory=(
        "You are an OCR expert. Your job is to meticulously scan the entire invoice PDF "
        "and convert all text, including tables and figures, into a raw text format. "
        "You are the first step in the data pipeline."
    ),
    tools=[ocr_extractor_tool], # Pass the function directly as a tool
    verbose=True,
    model="gpt-4o-mini"
)

'''llm_extraction_agent = Agent(
    role="LLM-based Data Extractor",
    goal="Extract specific data points from raw invoice text.",
    backstory=(
        "You are a highly skilled data extractor powered by a Large Language Model. "
        "You receive raw text from the OCR agent and your mission is to identify and "
        "extract key information like invoice number, date, total amount, and vendor "
        "details. You must be accurate and fast."
    ),
    #tools=[search_tool], # Uncomment and define search_tool if needed
    verbose=True,
    model="gpt-4o-mini"
)'''

llm_extraction_agent = Agent(
    role="LLM-based Data Extractor",
    goal="Accurately extract specific data points from raw invoice text.",
    backstory=(
       "You are a highly skilled and precise data extractor. Your mission is to "
    "identify and extract all key information for a structured JSON output. "
    "This includes the **invoice number**, **date**, **total amount**, **vendor details**, "
    "and **line items** or **description**. "
    "You must only extract values that are clearly labeled. "
    "For example, only extract an **invoice number** if the text explicitly "
    "says 'Invoice Number', 'Invoice #', or 'Invoice ID'. "
    "Similarly, only extract a **date** if it's labeled as 'Date', 'Invoice Date', or 'Due Date'. "
    "**Crucially, you must ignore any field labeled as 'Reference Number'** "
    "or similar, as this is not an invoice number."
),
    #tools=[search_tool], # Uncomment and define search_tool if needed
    verbose=True,
    model="gpt-4o-mini"
)

validation_agent = Agent(
    role="Data Validator",
    goal="Ensure the extracted data is accurate and complete.",
    backstory=(
        "You are a meticulous validator. Your task is to check the data extracted by the "
        "LLM against a set of predefined rules. If any data is missing or improperly "
        "formatted, you flag it for a retry."
    ),
    verbose=True,
    model="gpt-4o-mini"
)

json_builder_agent = Agent(
    role="JSON Formatter",
    goal="Transform validated data into a structured JSON format.",
    backstory=(
        "You are the final formatter. Your mission is to take the clean, validated data "
        "and build a perfect JSON object, ready for downstream systems."
    ),
    verbose=True,
    model="gpt-4o-mini"
)

In [81]:
# Define Tasks
task_ocr = Task(
    description=(
        "Use the extractor to read the invoice PDF located at '{pdf_path}' and extract "
        "all text content. Provide the raw text output to the next agent."
    ),
    expected_output="The complete, raw text content of the invoice, including all text from the PDF.",
    agent=ocr_agent
)

task_extract = Task(
    description=(
        "From the raw text, extract key fields. For 'invoice number', "
        "look for labels like 'invoice', 'invoice_number', or 'inv#'. "
        "For 'date', look for labels like 'date', 'invoice date', or 'due date'."
        "**DO NOT** extract any 'reference number' as an invoice number."
    ),
    expected_output="A JSON object containing the extracted fields.",
    agent=llm_extraction_agent
)

task_validate = Task(
    description=(
        "Validate the JSON output from the data extraction agent. "
        "Ensure the 'invoice number' and 'date' fields are present. "
        "If either is missing, flag the data as invalid. Do not validate 'reference number' as an invoice."
    ),
    expected_output="A JSON object with a 'status' key (valid or invalid) and the validated data.",
    agent=validation_agent
)

'''task_build_json = Task(
    description=(
        "Take the validated and corrected data from the previous steps and convert it into "
        "a final, clean JSON object. The JSON should only contain the extracted fields."
    ),
    expected_output="A valid JSON object containing the extracted invoice data.",
    agent=json_builder_agent
)'''

task_build_json = Task(
    description=(
        "Take the validated data from the previous step. "
        "If the input data's 'status' is 'invalid', your final output must be a JSON object "
        "like this: `{\"error\": \"Invalid input format. Missing required fields.\"}`. "
        "If the data is valid, convert it into a final, clean JSON object. "
        "The final JSON should only contain the extracted fields and no 'status' key."
    ),
    expected_output="A valid JSON object containing either the extracted invoice data or an error message.",
    agent=json_builder_agent
)

In [82]:
# ----------------------
# Crew Setup
# ----------------------

invoice_processing_crew = Crew(
    agents=[
        ocr_agent,
        llm_extraction_agent,
        validation_agent,
        json_builder_agent
    ],
    tasks=[
        task_ocr,
        task_extract,
        task_validate,
        task_build_json
    ],
    process=Process.sequential,
    verbose=True
)

In [83]:
# The code execution block
# This is where the process is initiated and the PDF path is passed.
def run_crew(pdf_path):
    print("Running the invoice processing crew...")
    result = invoice_processing_crew.kickoff(inputs={'pdf_path': pdf_path})
    print("\n✅ Final Output:")
    print(result)

# Example usage (uncomment and replace with your actual file path)
# if __name__ == "__main__":
#     from google.colab import files
#     uploaded = files.upload()
#     pdf_path = list(uploaded.keys())[0]
#     run_crew(pdf_path)

============================================================================

In [84]:
run_crew(r'/content/invoice-0-4.pdf')

Running the invoice processing crew...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2b2270e7-fea4-40b2-a539-f4583415f0aa                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: OCR Specialist                                                                                          │
│                                                                                                                 │
│  Task: Use the extractor to read the invoice PDF located at '/content/invoice-0-4.pdf' and extract all text     │
│  content. Provide the raw text output to the next agent.                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: JSON Formatter                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "invoice_number": "BPXINV-00550",                                                                            │
│    "date": "23.05.2021",                                                                                        │
│    "total_amount": 6610.95,                                                                                     │
│    "vendor_details": {                                                                                          │
│      "name": "Bioplex",                                                                                         │
│      "address": "5 Rue Bader, Narbonne, Aude, 11100",                                                           │
│      "phone": "+33 140 260294"                                                                                  │
│    },                                                                                                           │
│    "line_items": [                                                                                              │
│      {                                                                                                          │
│        "quantity": 10,                                                                                          │
│        "description": "Dextromethorphan polistirex",                                                            │
│        "unit_price": 12.45,                                                                                     │
│        "total": 124.50                                                                                          │
│      },                                                                                                         │
│      {                                                                                                          │
│        "quantity": 25,                                                                                          │
│        "description": "Venlafaxine Hydrochloride",                                                              │
│        "unit_price": 16.00,                                                                                     │
│        "total": 400.00                                                                                          │
│      },                                                                                                         │
│      {                                                                                                          │
│        "quantity": 25,                                                                                          │
│        "description": "Metoclopramide Hydrochloride (BPXPO-00537)",                                             │
│        "unit_price": 9.99,                                                                                      │
│        "total": 249.75                                                                                          │
│      },                                                                                                         │
│      {                                                                                                          │
│        "quantity": 10,                                 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 03236ecc-d6c9-493c-b173-9f97282c767b                                                                     │
│  Agent: JSON Formatter                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2b2270e7-fea4-40b2-a539-f4583415f0aa                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "invoice_number": "BPXINV-00550",                                                                            │
│    "date": "23.05.2021",                                                                                        │
│    "total_amount": 6610.95,                                                                                     │
│    "vendor_details": {                                                                                          │
│      "name": "Bioplex",                                                                                         │
│      "address": "5 Rue Bader, Narbonne, Aude, 11100",                                                           │
│      "phone": "+33 140 260294"                                                                                  │
│    },                                                                                                           │
│    "line_items": [                                                                                              │
│      {                                                                                                          │
│        "quantity": 10,                                                                                          │
│        "description": "Dextromethorphan polistirex",                                                            │
│        "unit_price": 12.45,                                                                                     │
│        "total": 124.50                                                                                          │
│      },                                                                                                         │
│      {                                                                                                          │
│        "quantity": 25,                                                                                          │
│        "description": "Venlafaxine Hydrochloride",                                                              │
│        "unit_price": 16.00,                                                                                     │
│        "total": 400.00                                                                                          │
│      },                                                                                                         │
│      {                                                                                                          │
│        "quantity": 25,                                                                                          │
│        "description": "Metoclopramide Hydrochloride (BPXPO-00537)",                                             │
│        "unit_price": 9.99,                                                                                      │
│        "total": 249.75                                                                                          │
│      },                                                                                                         │
│      {                                                


✅ Final Output:
{
  "invoice_number": "BPXINV-00550",
  "date": "23.05.2021",
  "total_amount": 6610.95,
  "vendor_details": {
    "name": "Bioplex",
    "address": "5 Rue Bader, Narbonne, Aude, 11100",
    "phone": "+33 140 260294"
  },
  "line_items": [
    {
      "quantity": 10,
      "description": "Dextromethorphan polistirex",
      "unit_price": 12.45,
      "total": 124.50
    },
    {
      "quantity": 25,
      "description": "Venlafaxine Hydrochloride",
      "unit_price": 16.00,
      "total": 400.00
    },
    {
      "quantity": 25,
      "description": "Metoclopramide Hydrochloride (BPXPO-00537)",
      "unit_price": 9.99,
      "total": 249.75
    },
    {
      "quantity": 10,
      "description": "Avobenzone, octinoxate (BPXPO-00538)",
      "unit_price": 4.45,
      "total": 44.50
    },
    {
      "quantity": 10,
      "description": "Verapamil hydrochloride",
      "unit_price": 7.89,
      "total": 78.90
    },
    {
      "quantity": 15,
      "description": "

In [85]:
run_crew(r'/content/Asim_donation_Zakat2.pdf')

Running the invoice processing crew...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2b2270e7-fea4-40b2-a539-f4583415f0aa                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: OCR Specialist                                                                                          │
│                                                                                                                 │
│  Task: Use the extractor to read the invoice PDF located at '/content/Asim_donation_Zakat2.pdf' and extract     │
│  all text content. Provide the raw text output to the next agent.                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: JSON Formatter                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"error": "Invalid input format. Missing required fields."}                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 03236ecc-d6c9-493c-b173-9f97282c767b                                                                     │
│  Agent: JSON Formatter                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2b2270e7-fea4-40b2-a539-f4583415f0aa                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {"error": "Invalid input format. Missing required fields."}                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


✅ Final Output:
{"error": "Invalid input format. Missing required fields."}
